# Resolucion prueba de ingreso - Facundo Kuzis

### Importo los paquetes necesarios e inicio la API de Rofex

In [1]:
import pyRofex
import datetime
import numpy as np
import pandas as pd
import yfinance as yf
from time import sleep
import warnings
warnings.filterwarnings("ignore")


pyRofex.initialize('fkuzis7851',
                   'szqraX8)',
                   'REM7851',
                   pyRofex.Environment.REMARKET)

## get_f_prices

In [ ]:
def get_f_prices(instrument):
    
    """
    Obtiene los precios bid y ask del instrumento especificado utilizando la biblioteca pyRofex.
    
    Input:
        instrument (str): El símbolo del instrumento financiero a utilizar.
        
    Returns:
        tuple: Una tupla con el precio bid y ask del contrato. Si no hay precios bid o ask disponibles,
        se devuelve un valor None para el precio correspondiente.
    """
    
    # Defino una lista con los tipos de datos que se van a solicitar en el mercado
    entries = [pyRofex.MarketDataEntry.BIDS, pyRofex.MarketDataEntry.OFFERS]
    
    # Realizo una solicitud de datos de mercado al servidor de Rofex con el instrumento y los tipos de datos especificados
    market_data = pyRofex.get_market_data(instrument, entries)
    
    # Extaigo el precio bid del objeto de datos de mercado recibido
    BI = market_data['marketData']['BI']
    if len(BI) == 0:
        # Si no hay precios bid disponibles, se devuelve un valor None
        bid = None
    else:
        # Si hay precios bid disponibles, se toma el primer precio de la lista de precios bid y se asigna a la variable 'bid'
        bid = BI[0]['price']
    
    # Se extrae el precio ask del objeto de datos de mercado recibido
    OF = market_data['marketData']['OF']
    if len(OF) == 0:
        # Si no hay precios ask disponibles, se devuelve un valor None
        ask = None
    else:
        # Si hay precios ask disponibles, se toma el primer precio de la lista de precios ask y se asigna a la variable 'ask'
        ask = OF[0]['price']
    
    # Se devuelve el precio bid y ask
    return bid, ask


## get_vencimiento

In [3]:

def get_vencimiento(instrumento):
    """
    Obtiene la fecha de vencimiento del instrumento especificado utilizando la biblioteca pyRofex.
    
    Input:
        instrumento (str): El símbolo del instrumento financiero a utilizar.
        
    Output:
        datetime.datetime: La fecha de vencimiento del contrato en formato datetime.
        
    """
    # Solicito los detalles del instrumento financiero
    info = pyRofex.get_instrument_details(instrumento)
    
    # Obtengo la fecha de vencimiento y convierto a formato datetime
    mat_date = info['instrument']['maturityDate']
    mat_date = datetime.datetime.strptime(mat_date, '%Y%m%d')

    return mat_date

## calcular_tasa_implicita

In [4]:

def calcular_tasa_implicita(precios):
    """
    Calcula la tasa implícita para los precios de un instrumento financiero dado 
    y devuelve un DataFrame con los resultados.
    
    Input:
        precios (pd.DataFrame): Un DataFrame que contiene los precios del instrumento financiero, incluyendo el precio
                                subyacente, el precio Ask y Bid del contrato, y la fecha de vencimiento.
        
    Output:
        pd.DataFrame: Un DataFrame que contiene los precios del contrato, así como las tasas implícitas
                          correspondientes al Bid y al Ask.
    """
    # Obtengo la fecha actual
    now = datetime.datetime.now()
        
    # Calculo la diferencia en días entre la fecha de vencimiento y la fecha actual, y convierto a años
    delta_days = precios.Vencimiento - now
    delta_years = [delta_days[i].days / 365 for i in range(len(delta_days))]
    delta_years = np.array(delta_years)
    
    # Calculo la tasa implícita correspondiente al Bid y al Ask y los agrego al DataFrame original
    precios['Implicita Bid'] = (precios.Bid / precios['Precio Subyacente'])**(1/delta_years) - 1
    precios['Implicita Ask'] = (precios.Ask / precios['Precio Subyacente'])**(1/delta_years) - 1
    
    # Devuelvo el DataFrame con los resultados
    return precios

## info_instrumentos

In [5]:
def info_instrumentos(lista_instrumentos):
    
    """
    Crea un DataFrame con información acerca de los instrumentos financieros dados.
    
    Input:
    lista_instrumentos (list): Una lista de strings con los tickers de los instrumentos.
    
    Output:
    pd.DataFrame: Un DataFrame con tres columnas:
        - Ticker: el ticker de cada instrumento.
        - Subyacente: el subyacente correspondiente a cada instrumento.
        - Vencimiento: la fecha de vencimiento correspondiente a cada instrumento.
    """
    
    # Creo dataframe donde se guardará la información de precios de los instrumentos
    precios = pd.DataFrame(columns=['Ticker'], data = lista_instrumentos)
       
    # Obtengo el nombre de cada subyacente a partir de las primeras 4 letras del nombre del instrumento
    # y le agrego el sufijo .BA
    subyacentes = [tick[:4] + '.BA' for tick in precios['Ticker']] 
    
    # Guardo los nombres de los subyacentes en una nueva columna en df precios
    precios['Subyacente'] = subyacentes
    
    # La lógica de que el subyacente es las 4 primeras letras solo funciona cuando son sobre acciones
    # por lo que le asigno el nombre correcto a los subyacentes correspondientes al dólar/peso
    precios['Subyacente'][precios['Subyacente'] == 'DLR/.BA'] = 'USDARS=X'
    
    
    # Para cada contrato, obtengo la fecha de vencimiento a partir de la función get_vencimiento
    vencimientos = []
    for ins in lista_instrumentos:
       v = get_vencimiento(ins)
       vencimientos.append(v)
        
    # Guardo las fechas en una nueva columna en el dataframe
    precios['Vencimiento'] = vencimientos
    
    # Devuelvo el dataframe con columna de Ticker, Subyacente y Vencimiento
    return(precios)

## get_prices

In [6]:
def get_prices(precios):

    """
     Actualiza un DataFrame de precios de mercado con precios bid y ask de los instrumentos financieros,
     precios spot de los activos subyacentes y tasas implícitas de bid y ask.
     
     Input:
         DataFrame con la información de los contratos a utilizar.
         
     Output:
         DataFrame con los precios de mercado actualizados.     
    """    

    # Creo una columna nueva para los precios Bid del instrumento, y otra para los ask   
    precios['Bid'] = None
    precios['Ask'] = None

    # Para cada uno de los instrumentos dados, obtengo los precios bid y ask a partir de la función get_f_prices
    for ins in lista_instrumentos:       
        bid, ask = get_f_prices(ins)
        # Se agergan los precios en el dataframe
        precios['Bid'][precios['Ticker'] == ins] = bid
        precios['Ask'][precios['Ticker'] == ins] = ask

    

    # Obtengo los nombres de todos los subyacentes, sin que se repitan
    subs_unicos = np.unique(precios.Subyacente)
    
    # Creo una nueva columna en el df para guardar los precios spot de los subyacentes
    precios['Precio Subyacente'] = None
    
    # Extraigo con yfinance el precio spot de cada uno de los subyacentes y los agrego al df precios
    for stock in subs_unicos:
        # Me quedo con el precio de cierre del último minuto disponible
        precio_sub = yf.download(stock, period = '1d', interval= '1m', progress=False)['Adj Close'][-1] 
        precios['Precio Subyacente'][precios['Subyacente'] == stock] = precio_sub

    #precios['Bid'][1] = 550.1


    # Utilizo la función calcular_tasa_implicita, que agrega columna de Implicita Bid, e Implicita Ask
    precios = calcular_tasa_implicita(precios)
    
    # Devuelvo el dataframe con la información del mercado actualizada
    return(precios)

## verificar_arbitraje

In [7]:
def verificar_arbitraje(precios, ganancia_t0 = True):
    """
    Verifica si existe un arbitraje a partir de los precios de los subyacentes y los contratos dados.
    En caso de que exista, calcula el arbitraje de mayor ganancia.
    El arbitraje buscado en este caso existe cuando se puede 'pedir prestado' a partir de shortear una acción hoy y
    recomprarla a futuro, a una tasa implicita menor que lo que se puede 'colocar' a partir de comprar una acción hoy y
    venderla a futuro. Es decir, existe arbitraje si hay una tasa bid implícita mayor a alguna tasa ask.
    
    Input:
    precios (DataFrame): Un DataFrame que contiene información de precios de los futuros, junto a los precios de los subyacentes, y las fechas de vencimientos.
    ganancia_t0 (bool, opcional): Si es True, calcula la ganancia en el momento 0. Si es False, calcula la ganancia en el momento de vencimiento. Valor por default: True.

    Returns:
    bool: Si existe o no un arbitraje.

    """

    # Calcular los spreads para cada vencimiento y obtener el vencimiento que contiene al mayor spread
    vencimientos_unique = np.unique(precios.Vencimiento)
    spreads = pd.DataFrame(columns=['Vencimiento', 'Spread'])
    for vencimiento in vencimientos_unique:
        # DataFrame temporal que contiene únicamente los contratos con este vencimiento determinado
        tmp = precios[precios['Vencimiento'] == vencimiento] 

    	# Obtengo mejor tasa Bid y mejor tasa Ask para este vencimiento
        max_bid_vencimiento = tmp['Implicita Bid'].max()
        min_ask_vencimiento = tmp['Implicita Ask'].min()
        
        # Calculo el mejor spread para este vencimiento
        spread = max_bid_vencimiento - min_ask_vencimiento
        spreads = spreads.append({'Vencimiento': vencimiento, 'Spread': spread}, ignore_index=True)

    # Defino cual es el vencimiento con mejor spread
    pos_max_spread = np.where(spreads['Spread'] == spreads['Spread'].max())[0][0]
    best_vencimiento = spreads['Vencimiento'][pos_max_spread]


    # Obtengo los precios para el mejor vencimiento
    tmp = precios[precios['Vencimiento'] == best_vencimiento]
    max_bid = tmp['Implicita Bid'].max()
    min_ask = tmp['Implicita Ask'].min()

    # Verificar que el spread es positivo. En caso contrario, no hay arbitraje.
    if max_bid < min_ask:
        print('No hay arbitraje posible')
        return False

    # Obtener los precios para los bid y ask con la mejor ganancia
    pos_max_bid = np.where(precios['Implicita Bid'] == max_bid)[0][0]
    pos_min_ask = np.where(precios['Implicita Ask'] == min_ask)[0][0]
    best_bid = precios.iloc[pos_max_bid]
    best_ask = precios.iloc[pos_min_ask]

    # Calcular la ganancia y la cantidad de spot a comprar, dependiendo de si se pida la ganancia en t0 o al vencimiento

    if ganancia_t0:
        cantidad_compra_spot = best_ask.Ask  / best_bid.Bid # Para netear los flujos al vencimiento
        ganancia = best_ask['Precio Subyacente'] - cantidad_compra_spot *   best_bid['Precio Subyacente']
        mensaje_ganancia = '\nGanancia en momento 0: $' + str(ganancia.round(3))
    else: 
        cantidad_compra_spot = best_ask['Precio Subyacente']  / best_bid['Precio Subyacente'] # Para netear los flujos de t0
        ganancia = (cantidad_compra_spot *  best_bid.Bid - best_ask.Ask)
        mensaje_ganancia = '\nGanancia en momento de vencimiento: $' + str(ganancia.round(3))

    # Imprimir los resultados
    print('Mejor arbitraje',
               
          '\nComprar spot:', best_bid.Subyacente, '@', best_bid['Precio Subyacente'].round(3), 'Cantidad:', round(cantidad_compra_spot,3), 
          'Flujo T0: -', round(cantidad_compra_spot *  best_bid['Precio Subyacente'],3), 
         
          '\nVender spot:', best_ask.Subyacente, '@', best_ask['Precio Subyacente'].round(3),  'Cantidad:', 1.0, 
          'Flujo T0: +', best_ask['Precio Subyacente'].round(3), 
               
          '\nVender a futuro:', best_bid.Ticker, '@', round(best_bid.Bid,3), 'Cantidad:', round(cantidad_compra_spot,3), 
          'Flujo T: +', round(cantidad_compra_spot *  best_bid.Bid,3), 
             
          '\nComprar a futuro:', best_ask.Ticker, '@', round(best_ask.Ask, 3), 'Cantidad:', 1.0, 
          'Flujo T: -', round(best_ask.Ask, 3), 
          
          mensaje_ganancia)
    
    return(True)


## Función principal 'main'

In [8]:
def main(lista_instrumentos, only_once = False):
   
    """
    Función principal del programa. Recibe una lista de instrumentos y un booleano 'only_once'. 
    Obtiene la información de los instrumentos, los precios de mercado, y calcula posibles arbitrajes.
    Si only_once es True, solo se ejecuta una vez y devuelve el DataFrame actualizado de precios. 
    Si only_once es False, se ejecuta indefinidamente cada 20 segundos.
    
    
    Input:
        lista_instrumentos (list[str]): Lista de instrumentos a monitorear
        only_once (bool, optional): Booleano que indica si se ejecuta solo una vez o no. 
                                    Por defecto es False.
    
    Output:
        pd.DataFrame: Retorna un DataFrame actualizado con la información de los instrumentos y sus precios, si se pide ejecutar solo una vez.
    """
   
    # Uso la función 'info_instrumentos' para obtener información sobre los instrumentos
    precios = info_instrumentos(lista_instrumentos)
    
    # Uso la función 'get_prices' para obtener los precios de los instrumentos y subyacentes
    precios = get_prices(precios)

    # Uso la función 'verificar_arbitraje' para detectar oportunidades de arbitraje, e imprimirlas en la consola si las hay
    arbitraje = verificar_arbitraje(precios)

    # Imprimo la hora actual
    print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

    # Si se pedía ejecutar solo una vez, devuelve el DataFrame actualizado y termina el programa
    if only_once:
        return precios

    # Si 'only_once' es False, comienza un loop que se ejecutará indefinidamente
    while(True):
        
        # Esperar 20 segundos antes de continuar con la ejecución
        sleep(20)
        
        # Actualizo los precios
        precios_nuevos = get_prices(precios)
        
        # Inicio un booleano que tomará el valor True si hay un cambio en los precios
        cambio = False
    
        # Verifico si hay algún cambio en los precios, que se vería reflejado en un cambio en las tasas implícitas
        for i in range(len(precios_nuevos)):
 
            # Si cambia la tasa implícita del bid
            if (precios_nuevos['Implicita Bid'][i] != precios['Implicita Bid'][i]):
            
                # Si al menos uno de los dos no es NA
                if not (pd.isna(precios_nuevos['Implicita Bid'][i]) and (pd.isna(precios['Implicita Bid'][i]))):
               
                    # Imprimo el cambio de tasa
                    print('Cambio de tasa implicita bid:', precios_nuevos['Ticker'][i], 'de', precios['Implicita Bid'][i], 'a', precios_nuevos['Implicita Bid'][i])
                   
                    # Actualizo el booleano 'cambio'
                    cambio = True
    
            # Si cambia la tasa implícita del ask
            if (precios_nuevos['Implicita Ask'][i] != precios['Implicita Ask'][i]):
               
                # Si al menos uno de los dos no es NA
                if not (pd.isna(precios_nuevos['Implicita Ask'][i]) and (pd.isna(precios['Implicita Ask'][i]))):
        
                    # Imprimo el cambio de tasa
                    print('Cambio de tasa implicita ask:', precios_nuevos['Ticker'][i], 'de', precios['Implicita Ask'][i], 'a', precios_nuevos['Implicita Ask'][i])
                    
                    # Actualizo el booleano 'cambio'
                    cambio = True
            
        # Si hubo algún cambio de precios vuelvo a verificar si hay arbitraje. En caso contrario, no hace falta.
        if cambio:
            arbitraje = verificar_arbitraje(precios)
        else:
            print('Sin cambios')
    
        # Actualizo el DataFrame precios para la próxima vuelta del loop
        precios = precios_nuevos.copy()
        
        # Imprimo la hora actual
        print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

## Ejemplo única ejecución

In [13]:
lista_instrumentos = ['GGAL/ABR23','GGAL/JUN23','PAMP/ABR23','PAMP/JUN23','YPFD/ABR23','YPFD/JUN23', 'DLR/ABR23', 'DLR/JUN23']
precios = main(lista_instrumentos, True)

precios

No hay arbitraje posible
2023-03-12 16:08:06


,Ticker,Subyacente,Vencimiento,Bid,Ask,Precio Subyacente,Implicita Bid,Implicita Ask
0,GGAL/ABR23,GGAL.BA,2023-04-28,None,511.7,465.0,NaN,1.136924
1,GGAL/JUN23,GGAL.BA,2023-06-30,None,588.7,465.0,NaN,1.203112
2,PAMP/ABR23,PAMP.BA,2023-04-28,484.7,535.3,472.0,0.234514,1.714423
3,PAMP/JUN23,PAMP.BA,2023-06-30,None,545.3,472.0,NaN,0.621575
4,YPFD/ABR23,YPFD.BA,2023-04-28,None,None,4233.0,NaN,NaN
5,YPFD/JUN23,YPFD.BA,2023-06-30,None,None,4233.0,NaN,NaN
6,DLR/ABR23,USDARS=X,2023-04-28,197.9,225.0,200.729996,-0.10655,1.473598
7,DLR/JUN23,USDARS=X,2023-06-30,None,256.9,200.729996,NaN,1.284605


## Ejemplo loop infinito

In [22]:
lista_instrumentos = ['GGAL/ABR23','GGAL/JUN23','PAMP/ABR23','PAMP/JUN23','YPFD/ABR23','YPFD/JUN23', 'DLR/ABR23', 'DLR/JUN23']
precios = main(lista_instrumentos)


No hay arbitraje posible
2023-03-12 16:14:26
Sin cambios
2023-03-12 16:14:50
Sin cambios
2023-03-12 16:15:14
Sin cambios
2023-03-12 16:15:38
Sin cambios
2023-03-12 16:16:03
Sin cambios
2023-03-12 16:16:26


KeyboardInterrupt: 

## Ejemplo arbitraje encontrado

In [18]:
precios_ejemplo = precios.copy()
precios_ejemplo['Bid'][1] = 550.1
precios_ejemplo = calcular_tasa_implicita(precios_ejemplo)
precios_ejemplo

,Ticker,Subyacente,Vencimiento,Bid,Ask,Precio Subyacente,Implicita Bid,Implicita Ask
0,GGAL/ABR23,GGAL.BA,2023-04-28,None,511.7,465.0,NaN,1.136924
1,GGAL/JUN23,GGAL.BA,2023-06-30,550.1,588.7,465.0,0.755544,1.203112
2,PAMP/ABR23,PAMP.BA,2023-04-28,484.7,535.3,472.0,0.234514,1.714423
3,PAMP/JUN23,PAMP.BA,2023-06-30,None,545.3,472.0,NaN,0.621575
4,YPFD/ABR23,YPFD.BA,2023-04-28,None,None,4233.0,NaN,NaN
5,YPFD/JUN23,YPFD.BA,2023-06-30,None,None,4233.0,NaN,NaN
6,DLR/ABR23,USDARS=X,2023-04-28,197.9,225.0,200.729996,-0.10655,1.473598
7,DLR/JUN23,USDARS=X,2023-06-30,None,256.9,200.729996,NaN,1.284605


In [19]:
verificar_arbitraje(precios_ejemplo) # Con ganancia en t0

Mejor arbitraje 
Comprar spot: GGAL.BA @ 465.0 Cantidad: 0.991 Flujo T0: - 460.943 
Vender spot: PAMP.BA @ 472.0 Cantidad: 1.0 Flujo T0: + 472.0 
Vender a futuro: GGAL/JUN23 @ 550.1 Cantidad: 0.991 Flujo T: + 545.3 
Comprar a futuro: PAMP/JUN23 @ 545.3 Cantidad: 1.0 Flujo T: - 545.3 
Ganancia en momento 0: $11.057


True

In [20]:
verificar_arbitraje(precios_ejemplo, False) # Con ganancia al vencimiento

Mejor arbitraje 
Comprar spot: GGAL.BA @ 465.0 Cantidad: 1.015 Flujo T0: - 472.0 
Vender spot: PAMP.BA @ 472.0 Cantidad: 1.0 Flujo T0: + 472.0 
Vender a futuro: GGAL/JUN23 @ 550.1 Cantidad: 1.015 Flujo T: + 558.381 
Comprar a futuro: PAMP/JUN23 @ 545.3 Cantidad: 1.0 Flujo T: - 545.3 
Ganancia en momento de vencimiento: $13.081


True

## Test unitario para función que calcula tasa

In [21]:
def test_calcular_tasa_implicita():
    
    # Creo un DataFrame con datos ficticios para las pruebas
    precios = pd.DataFrame({
        'Vencimiento': [datetime.datetime.now() + datetime.timedelta(days=365), 
                        datetime.datetime.now() + datetime.timedelta(days=730)],
        'Bid': [99.5, 100.5],
        'Ask': [100.0, 101.0],
        'Precio Subyacente': [100.0, 100.0]
    })
    
    # Llamo a la función que se está probando
    precios_calculados = calcular_tasa_implicita(precios)
    
    # Compruebo que se hayan agregado las columnas 'Implicita Bid' e 'Implicita Ask'
    assert 'Implicita Bid' in precios_calculados.columns
    assert 'Implicita Ask' in precios_calculados.columns
    
    # Calculo por mi cuenta las tasas implícitas
    t_bid = (precios.Bid / precios['Precio Subyacente'])** np.array([1, 1/2]) - 1
    t_ask = (precios.Ask / precios['Precio Subyacente'])** np.array([1, 1/2]) - 1
    
    # Tolerancia de error absoluto
    atol = 1e-4
    
    # Compruebo que los valores de las nuevas columnas sean correctos
    assert np.isclose(precios_calculados['Implicita Bid'][0], t_bid[0], atol=atol)
    assert np.isclose(precios_calculados['Implicita Ask'][0], t_ask[0], atol=atol)
    assert np.isclose(precios_calculados['Implicita Bid'][1], t_bid[1], atol=atol)
    assert np.isclose(precios_calculados['Implicita Ask'][1], t_ask[1], atol=atol)
    
    # Si no hay ningún error, imprime el resultado
    print('Test superado')
    
test_calcular_tasa_implicita()

Test superado
